In [1]:
import numpy as np

In [2]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [12,12]


def fix_scaling(ax=None):
    if not ax:
        xlim = plt.xlim()
        ylim = plt.ylim()
        d1 = xlim[1] - xlim[0]
        d2 = ylim[1] - ylim[0]
        if d1 > d2:
            plt.ylim((ylim[0] - (d1-d2) / 2, ylim[1] + (d1-d2) / 2))
        else:
            plt.xlim((xlim[0] + (d1-d2) / 2, xlim[1] - (d1-d2) / 2))
    else:
        xlim = ax.get_xlim()
        ylim = ax.get_ylim()
        d1 = xlim[1] - xlim[0]
        d2 = ylim[1] - ylim[0]
        if d1 > d2:
            ax.set_ylim((ylim[0] - (d1-d2) / 2, ylim[1] + (d1-d2) / 2))
        else:
            ax.set_xlim((xlim[0] + (d1-d2) / 2, xlim[1] - (d1-d2) / 2))

In [3]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML


def animate_trajectory(trajectories, n, f, x_min):
    fig, ax = plt.subplots()
    colors = ['blue']
    
    def step(t):
        ax.cla()
        ax.plot([x_min[0]], [x_min[1]], 'o', color='green')
        ax.add_patch(plt.Rectangle((0, 0), 1, 1))
        # Level contours
        delta = 0.025
        x = np.arange(-3, 3, delta)
        y = np.arange(-3, 3, delta)
        X, Y = np.meshgrid(x, y)
        Z = np.zeros_like(X)
        # print(X.shape, Y.shape)
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                Z[i][j] = f([X[i][j], Y[i][j]])
        CS = ax.contour(X, Y, Z, [0.5, 3, 7.5], colors=['blue', 'purple', 'red'])

        for traj, color in zip(trajectories, colors):
            ax.plot([u[0] for u in traj[0][:t]], [u[1] for u in traj[0][:t]], color=color)
            ax.plot([u[0] for u in traj[0][:t]], [u[1] for u in traj[0][:t]], 'o', color=color)

        fix_scaling(ax)
        ax.axis('off')

    plt.close()
    return FuncAnimation(fig, step, frames=range(n), interval=600)

In [4]:
def f(x):
    return 3 * x[0] ** 2 + 22 * (x[0] - x[1]) ** 2 - x[0] - 2 * x[1]


def f_grad(x):
    return np.array([50 * x[0] - 44 * x[1] - 1,
                     - 44 * x[0] + 44 * x[1] - 2], dtype=np.float32)


def f_hessian():
    return np.array([[50, -44],
                     [-44, 44]], dtype=np.float32)


def hessian_eigen_values():
    return list(np.linalg.eig(f_hessian()))[0]

In [5]:
import math


BOUNDS_EPS = 1e-9


# A, b = g()
# Ax + b <= 0
def g():
    return np.array([[-1, 0],
                     [1, 0],
                     [0, -1],
                     [0, 1]], dtype=np.float32), \
           np.array([0.7, -2, 0.5, -2], dtype=np.float32)


def F(f, x, t, A, b):
    y = A @ x + b
    return t * f(x) + sum([-math.log2(-min(yi, -BOUNDS_EPS)) for yi in y])


def F_grad(f_grad, x, t, A, b):
    y = A @ x + b
    return t * f_grad(x) + sum(np.array([1.0 / min(yi, -BOUNDS_EPS) * Ai for Ai, yi in zip(A, y)],
                                        dtype=np.float32))


def F_hessian(f_hessian, x, t, A, b):
    y = A @ x + b
    d = np.diag(np.array([1.0 / min(yi, -BOUNDS_EPS) ** 2 for yi in y],
                         dtype=np.float32))
    return t * f_hessian() + A.T @ d @ A

In [6]:
# Attempts to find interior point using auxiliary problem
# similar to simplex method first phase
def find_interior_point_auxiliary(g):
    A, b = g()
    n = len(A[0])
    m = len(A)
    # Building new constraints
    I = np.diag([1 if bi > 0 else 0 for bi in b])
    A_extended = np.concatenate((A, -I), axis=1)
    I = np.diag(np.repeat([1], [m]))
    A_extended = np.concatenate((A_extended, 
                                 np.concatenate((np.zeros((m, n)),
                                                 -I), 
                                                axis=1)),
                                axis=0)
    b_extended = np.concatenate((b, np.zeros(m)), axis=0)
    
    # Minimizing sum of additional parameters
    def new_grad(x):
        return np.concatenate((np.zeros(n), [1 if bi > 0 else 0 for bi in b]), axis=0)
    def new_hessian():
        return np.zeros((n + m, n + m))
    def new_g():
        return A_extended, b_extended

    # Feasible starting point for auxiliary problem
    x_start = np.concatenate((np.zeros(n), 
                              np.array([bi if bi > 0 else 0 for bi in b])),
                             axis=0)
    
    traj_int_point = interior_point(new_grad, new_hessian, new_g, x_start)
    int_point = traj_int_point[0][-1]
    extra_parameters = np.array(int_point[n:-1])
    eps = 1e-4
    if np.any([t > eps for t in extra_parameters]):
        return None
    
    return int_point[0:n]

In [7]:
from scipy import optimize


def find_interior_point(g):
    A, b = g()
    n = len(A[0])
    c = np.zeros(n)
    eps = 1e-4
    bounds = np.repeat([None], [n * 2])
    bounds = bounds.reshape((n, 2))
    # Finding any feasible point
    res = optimize.linprog(c=c, A_ub=A, b_ub=-(b + eps), bounds=bounds)
    
    if res.status == 2:
        # 2 : Problem appears to be infeasible
        return None
    
    point = res.x
    return point


def interior_point(f_grad, f_hessian, g, x_start=None):
    if x_start is None:
        x_start = find_interior_point(g)
    
    alpha = 2
    t = 0.1
    A, b = g()
    
    traj_int_point = [x_start.copy()]
    cur_x = x_start.copy()
    for i in range(NUMBER_OF_STEPS):
        dx = np.linalg.solve(F_hessian(f_hessian, cur_x, t, A, b),
                             F_grad(f_grad, cur_x, t, A, b))
        cur_x -= dx
        t *= alpha
        traj_int_point.append(cur_x.copy())

    print(traj_int_point[-1])
    return [traj_int_point, 'Interior point']

In [8]:
def solve_equation():
    zeros = np.zeros(2)
    x_min = np.linalg.solve(f_hessian(), -f_grad(zeros))
    print(x_min)
    # print(f(x_min))
    return x_min, f(x_min)

In [21]:
NUMBER_OF_STEPS = 40

In [22]:
trajectories = []
x_start = find_interior_point(g)
print(x_start)
trajectories.append(interior_point(f_grad, f_hessian, g, x_start))
x_min, f_min = solve_equation()

[ 0.7001      0.50010002]
[ 0.69999967  0.74545422]
[ 0.5         0.54545456]


In [18]:
base_animation = animate_trajectory(trajectories, NUMBER_OF_STEPS, f, x_min)
HTML(base_animation.to_html5_video())